# M1. Actividad

Humberto Genaro Cisneros Salinas A01723264
Agosto 17, 2025


## Conocer y aplicar una herramienta para la implementación de sistemas multiagentes.

A lo largo de este Notebook se describirá el razonamiento detrás de una solución para la modelación de un ambiente de cuadrícula en donde un agente decide su movimiento en una de 8 direcciones posibles y si pasa por una celda declarada como sucia, la limpia. Para esta solución se justificarán las decisiones detrás de su desarrollo y cómo esto afecta el comportamiento de la simulación.

### Instalación de Librerías

Esta sección solo se deberá correr una vez por ambiente en donde el notebook esté alojado.

In [ ]:
!pip install agentpy
!pip install seaborn

### Importación de las librerías Instaladas

Aunque seaborn no se utilice para esta solución se deja como referencia para futuros proyectos el bloque de importaciones. Se espera que este bloque se mantenga a lo largo de las prácticas a excepción de cuando se introduzcan nuevas librerías.

In [3]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import HTML
import agentpy as ap

## Diseño de Cuadrícula

El ambiente en el que se llevará acabo la simulación será una cuadrícula cuyo tamaño se define en los parámetros del modelo. Se sabe que existirán dos tipos de agentes en la simulación, el agente de limpieza y el agente de celda. Los agentes de celda representan cada uno una celda de la cuadrícula. Ambos agentes son puramente reactivos, en donde el agente de limpieza tiene la tarea de recordar su posición y moverse en una cierta dirección aleatoria, mientras que el agente de celda se encarga de saber si está limpio, sucio o en proceso de limpieza cuando un agente de limpieza está encima de este.

- Los agentes de limpieza son de color negro siempre
- Los agentes de celda son: blancos cuando están limpios, verdes cuando están sucios y negros cuando están en proceso de limpieza para no sobrescribir el color de los agentes de limpieza.

In [ ]:
def my_plot(model, ax):
    grid = np.ones((*model.environment.shape, 3))  # Emipeza como blanco

    for agent, pos in model.environment.positions.items():
        if agent.__class__.__name__ == "CleanerAgent":
            grid[pos] = [0, 0, 0]  
        elif agent.__class__.__name__ == "FloorCell":
            if agent.cleaning:
                grid[pos] = [0, 0, 0] # black
            elif agent.clean:
                grid[pos] = [1, 1, 1] #white
            else:
                grid[pos] = [0, 1, 0] #green
    
    ax.imshow(grid)

## Diseño de Agentes

Ambos agentes utilizanlos modelos de "Agent" de AgentPy

### Agente de Limpieza

El agente de limpieza recibe el ambiente por el que se mueve y empieza con una dirección neutral en el momento del setup.

La función de move_to_cell utiliza la librería de numpy para elegir de manera aleatoria una de ocho direcciones a la cual moverse. Como agentpy usa tuplas para definir el movimiento horizontal y vertical el resultado de esta función se manda directamente al ambiente con la función de move_by. El ambiente de agentpy determinará automáticamente si es posible moverse en esta dirección y de no ser posible el agente se mantendrá en su posición original.

In [ ]:
class CleanerAgent(ap.Agent):

    def setup(self, env):
        self.direction = (0,0) #Dirección inicial
        self.env = env #Ambiente
    
    def move_to_cell(self):
        directions = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]
        return directions[np.random.randint(0, len(directions), size=1)[0]]

    def execute(self):
        self.direction = self.move_to_cell()
        self.env.move_by(self, self.direction)

## Agente de Celda "de Piso"

Se crea uno de estos agentes para la cantidad de celdas que existen en el ambiente. Esto puede ser de gran ayuda para que en caso de escalar la solución a cuando existan obstáculos o comportamientos diferentes de la superficie en diferentes partes del ambiente, se pueda implementar directamente en el agente de FloorCell.

Cada agente recibe el ambiente en el que va a estar posicionado, su posición actual (la cual no va a cambiar), si está limpio o no y una lista que sirve de valor global para manipular la cantidad de celdas sucias que hay en el ambiente. La propiedad de que el agente está en proceso de limpieza, es decir que está en la misma posición que un agente de limpieza empieza como falsa y después se revisa si existe un agente del tipo CleanerAgent en la misma posición.

Por cada paso de ejecución, si la celda está en proceso de limpieza y no está limpia, se baja el contador de celdas sucias restantes y se marca la celda como limpia. Si no, se revisa si un agente de limpieza está en la misma posición para ser marcado como que está en proceso de limpieza.

In [ ]:
class FloorCell(ap.Agent):
    def setup(self, env, loc, clean, dirtyCellsContainer):
        self.env = env
        self.location = loc
        self.clean = clean
        self.cleaning = False
        self.dirtyCellsContainer = dirtyCellsContainer

        for agent, pos in self.env.positions.items():
            if isinstance(agent, CleanerAgent) and pos == self.location:
                self.cleaning = True
                break

    def execute(self):
        if (self.cleaning):
            
            if(not self.clean):
                self.dirtyCellsContainer[0] -= 1 #Bajan las celdas sucias restantes

            self.cleaning = False
            self.clean = True

            if(self.p.print):
                print("Cleaning Cell at {}".format(self.location))

        else:
            for agent, pos in self.env.positions.items(): #Revisa si está en proceso de limpieza
                if isinstance(agent, CleanerAgent) and pos == self.location:
                    self.cleaning = True
                    break



## Diseño de Modelo

### Setup


#### Inicialización de Variables
- Se inicializan las variables m y n para que estos parámetros tengan nombres más cortos y el código sea más legible.
- Se inicializa un contador de pasos que tomaron los agentes en limpiar todo el ambiente.
- Se obtiene el número de celdas a limpiar según el porcentaje de celdas sucias ingresado en los parámetros (Redondeado hacia abajo). Después se agrega a una lista para poder ser compartido "por referencia" a los agentes de celda.
- Se usa numpy para generar la cantidad de celdas sucias previamente establecida en forma de tuplas y se crea una lista con estas tuplas.
- Se crea el ambiente del tipo Grid o cuadrícula de agentpy para la simulación de tamaño MxN.

#### Agente de Limpieza
Se crea un alista con los agentes de limpieza que se establecieron en los parámetros del modelo y se agregan a la simulación comenzando en la posición 1,1 como se instruye en la actividad. Para esto se utilizó la forma abreviada de crear listas con ciclos for que tiene Python. Esto sirve para cuando son acciones sencillas y que se entienden directamente, pero no se debe usar para lógica más compleja como la de los agentes de celda.

#### Agente de Celda
Si la coordenada actual está dentro de la lista de celdas sucias que requieren limpieza, se inicializa un agente de celda sucio, sino se inicializa un agente de celda limpio. Todos los agentes se agregan a una lista y en el mismo ciclo se crea la lista de posiciones de cada agente para no tener que volver a correr un ciclo de complejidad O(M*N) solo para obtener todas las coordenadas del ambiente. Estos agentes se agregan a la simulación para terminar con el Setup del modelo.

## Step

Para cada paso de la simulación, si todavía quedan celdas por limpiar se le agrega uno al contador de pasos necesarios para limpiar todo el ambiente. Si se declaró el parámetro de print como verdadero, se dará información sobre la cantidad de celdas sucias restantes o, si ya no quedan, los pasos necesarios para limpiar toda la cuadrícula.

In [58]:
class CleanerModel(ap.Model):

    def setup(self):
        m = self.p.m
        n = self.p.n
        self.stepsToClean = 0
        dirtyCellNum = m*n*self.p.dirtyPercent//100
        self.dirtyCellsContainer = [dirtyCellNum]

        randomM = np.random.randint(0, m, size=dirtyCellNum)
        randomN = np.random.randint(0, n, size=dirtyCellNum)
        self.dirtyCells = list(zip(randomM, randomN))

        self.environment = ap.Grid(self, (m, n))

        self.cleanerAgents = [CleanerAgent(self, self.environment) for i in range(self.p.cleanerNum)]
        self.environment.add_agents(self.cleanerAgents, positions=[(1,1) for i in range(self.p.cleanerNum)])

        self.floorCellAgents = []
        cellPositions = []

        for i in range(n):
            for j in range(m):
                clean = (j,i) not in self.dirtyCells
                self.floorCellAgents.append(FloorCell(self, self.environment, (j,i),clean, self.dirtyCellsContainer))
                cellPositions.append((j,i))

        self.environment.add_agents(self.floorCellAgents, positions=cellPositions)

    def step(self):
        
        self.environment.agents.execute()
        if(self.dirtyCellsContainer[0] != 0):
            self.stepsToClean += 1

        if(self.p.print):
            print("\n********************")
            print("Remaining Dirty Cells: {}".format(self.dirtyCellsContainer[0]))

            if(self.dirtyCellsContainer[0] == 0):
                print("Steps to Clean: {}".format(self.stepsToClean))

## Implementación y Resultados de la Solución

- Se utiliza matplotlib para crear una figura y los ejes en los que existirá el ambiente del modelo.
- Se establecen los parámetros personalizables de la simulación en un diccionario, en el que se deberán respetar los nombres específicos.
- Se inicializa el modelo con los parámetros previos.
- La función de animate de agentpy recibe un modelo, una figura, los ejes de dicha figura y la representación específica del ambiente que se definió al principio del notebook para poder crear una animación utilizando matplot lib. Esta simulación se puede exportar como un archivo mp4 o gif o visualizar dentro del notebook utilizando HTML utilizando la función to_jshtml().

La cantidad de pasos puede variar, pero si se quiere llegar a apreciar el ambiente limpio se tendrá que usar un número de pasos mayor a la cantidad de celdas (m*n). Qué tan mayor dependerá de la cantidad de agentes de limpieza (cleanerNum) en la simulación.

In [63]:
fig, ax = plt.subplots()
parameters = {'print': True, 'steps':50, 'm':5, 'n':5, 'dirtyPercent': 10, 'cleanerNum':3 } 
cleanerModel = CleanerModel(parameters)
animation = ap.animate(cleanerModel, fig, ax, my_plot)
HTML(animation.to_jshtml())

Cleaning Cell at (1, 1)

********************
Remaining Dirty Cells: 2
Cleaning Cell at (0, 0)
Cleaning Cell at (1, 2)
Cleaning Cell at (2, 2)

********************
Remaining Dirty Cells: 2
Cleaning Cell at (0, 1)
Cleaning Cell at (1, 3)
Cleaning Cell at (3, 3)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (0, 0)
Cleaning Cell at (0, 2)
Cleaning Cell at (2, 4)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (3, 4)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (0, 1)
Cleaning Cell at (0, 2)
Cleaning Cell at (2, 4)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (0, 3)
Cleaning Cell at (1, 3)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (0, 1)
Cleaning Cell at (1, 2)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (0, 2)
Cleaning Cell at (2, 3)
Cleaning Cell at (1, 4)

********************
Remaining Dirty Cells: 1
Cleaning Cell at (1, 2)
Cleaning Cell at (1, 3)
Cleaning Cell at (2, 4)



# Uso de Chat GPT

## Prompts
"
```python
def my_plot(model, ax):
    grid = np.zeros(model.environment.shape)
    print(model.environment.positions)
    for agent, pos in model.environment.positions.items():
        grid[pos] = agent.id
    ax.imshow(grid, cmap='Greys')
```
I have two types of agents.
The agent of class Agent1 never changes color and is black.
The agent of class Agent2 has an attribute called clean.
If Agent2 is clean, make its color white, if it is not clean, make its color brown.
"

## Cambios
Modifiqué la respuesta de ChatGPT porque no había considerado la lógica de cuando el agente de limpieza estuviera "encima" de una celda.
En ese caso, no se mostraba el color negro del agente.